In [1]:
# importing all modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import math
import numpy as np
from osgeo import gdal
import osr
from IPython.display import display, clear_output
%matplotlib inline



# function to read in a data cube from a geo tiff file
def geotiff_to_datacube(fname):
    
    ds = gdal.Open(fname)
    
    geotransform = ds.GetGeoTransform()
    
    proj = osr.SpatialReference(wkt=ds.GetProjection())
    epsg = int(proj.GetAttrValue('AUTHORITY',1))
    
    xy_shape = np.array(ds.GetRasterBand(1).ReadAsArray()).shape
    
    # get number of bands in raster file
    n_bands = ds.RasterCount
    
    # initialize a data cube
    xyz_shape = xy_shape + (n_bands,)
    data_cube = np.ndarray(xyz_shape)
    
    # fill it with bands
    for i in range(1,n_bands+1):
        data_cube[:,:,i-1] =  np.array(ds.GetRasterBand(i).ReadAsArray())
    
    return data_cube, geotransform, epsg
    # end of read in datacube function


# global variables
rois = ['roi1','roi2','roi3']

folder_as = f'arealstatistik/'
folder_coefficients = f'data/coefficients/'
folder_composites = f'data/composites/'
folder_output = f'data/classification_data/'

spectral_bands = ['blue','green','red','nir','swir1','swir2']

yearAS = 2004 # 2004 or 2013
        

In [ ]:
# label coefficients

# creating name of features
coefficients = ['c','a1','b1']

kernelv = ['top','mid','lower']
kernelh = ['left','center','right']
positions = [f'{v}{h}' for v in kernelv for h in kernelh]

feature_names = [f'{pos}_{band}_{coef}' for pos in positions for band in spectral_bands for coef in coefficients]

# initializing containers
x_coords, y_coords, land_covers = ([],[],[])
datasets = []


for roi in rois:
    
    print(roi)
    
    # year of data acquisition
    if yearAS == 2004:
        year = 2006 if roi=='roi2' else 2007
    elif yearAS == 2013:
        year = 2015 if roi=='roi2' else 2016
    
    # loading arealstatistik
    df = pd.read_csv(f'{folder_as}{roi}_as{yearAS}_preprocessed.csv')
    nrows = df.shape[0]

    # loading coefficients
    dc_coefficients, geotransform, epsg = geotiff_to_datacube(f'{folder_coefficients}coefficients_{roi}_{year}.tif')

    # using lists and dictionary to avoid slow iteration with pd data frame
    x_list = list(df['X']); x_coords.extend(x_list)
    y_list = list(df['Y']); y_coords.extend(y_list)
    lc_list = list(df['land_cover']); land_covers.extend(lc_list)
    
    features = np.zeros((nrows,len(feature_names)))

    # unpack geotranform
    xOrigin = geotransform[0]
    yOrigin = geotransform[3]
    pixelWidth = geotransform[1]
    pixelHeight = -geotransform[5]
    
    for i, (x_coord,y_coord) in enumerate(zip(x_list,y_list)):
    

        # computing column and row indices
        icol = int((x_coord - xOrigin) / pixelWidth)
        irow = int((yOrigin - y_coord ) / pixelHeight)
    
        kernel = dc_coefficients[irow-1:irow+2,icol-1:icol+2,:]
        features[i,:] = kernel.flatten()
        
    dataset = pd.DataFrame(data=features,columns=feature_names)
    dataset['roi'] = roi
    datasets.append(dataset)
    

data = pd.concat(datasets,axis=0)
data['X'] = x_coords
data['Y'] = y_coords
data['land_cover'] = land_covers

# rearrange columns
data = data[['roi','X','Y','land_cover',*feature_names]]

data.to_csv(f'{folder_output}coefficients_labeled_as{yearAS}.csv', encoding='utf-8', index=False)
data.head()
        

In [ ]:
# label annual composites

# creating name of features
feature_names = spectral_bands

# initializing containers
x_coords, y_coords, land_covers = ([],[],[])
datasets = []

for roi in rois:
    
    print(roi)
    
    # year of data acquisition
    if yearAS == 2004:
        year = 2006 if roi=='roi2' else 2007
    elif yearAS == 2013:
        year = 2015 if roi=='roi2' else 2016
    
    # loading arealstatistik
    df = pd.read_csv(f'{folder_as}{roi}_as{yearAS}_preprocessed.csv')
    nrows = df.shape[0]

    # loading coefficients
    dc_composite, geotransform, epsg = geotiff_to_datacube(f'{folder_composites}annual_composite_{roi}_{year}.tif')

    # using lists and dictionary to avoid slow iteration with pd data frame
    x_list = list(df['X']); x_coords.extend(x_list)
    y_list = list(df['Y']); y_coords.extend(y_list)
    lc_list = list(df['land_cover']); land_covers.extend(lc_list)
    
    features = np.zeros((nrows,len(feature_names)))

    # unpack geotranform
    xOrigin = geotransform[0]
    yOrigin = geotransform[3]
    pixelWidth = geotransform[1]
    pixelHeight = -geotransform[5]
    
    for i, (x_coord,y_coord) in enumerate(zip(x_list,y_list)):
    
        # computing column and row indices
        icol = int((x_coord - xOrigin) / pixelWidth)
        irow = int((yOrigin - y_coord ) / pixelHeight)
    
        features[i,:] = dc_composite[irow,icol,:]

        
    dataset = pd.DataFrame(data=features,columns=feature_names)
    dataset['roi'] = roi
    datasets.append(dataset)
    

data = pd.concat(datasets,axis=0)
data['X'] = x_coords
data['Y'] = y_coords
data['land_cover'] = land_covers

# rearrange columns
data = data[['roi','X','Y','land_cover',*feature_names]]

data.to_csv(f'{folder_output}annual_composite_labeled_as{yearAS}.csv', encoding='utf-8', index=False)
data.head()
        

In [ ]:
# label annual composites

# creating name of features
seasons = ['spring','summer','autumn']
feature_names = [f'{band}_{season}' for pos in positions for band in spectral_bands]

# initializing containers
x_coords, y_coords, land_covers = ([],[],[])
datasets = []


for roi in rois:
    
    print(roi)
    
    # year of data acquisition
    if yearAS == 2004:
        year = 2006 if roi=='roi2' else 2007
    elif yearAS == 2013:
        year = 2015 if roi=='roi2' else 2016
    
    # loading arealstatistik
    df = pd.read_csv(f'{data_folder_as}{roi}_as{yearAS}_preprocessed.csv')
    nrows = df.shape[0]

    # loading coefficients
    dc_spring, geotransform, epsg = geotiff_to_datacube(f'{folder_composites}seasonal_composite_spring_{roi}_{year}.tif')
    dc_summer, _, _ = geotiff_to_datacube(f'{folder_composites}seasonal_composite_summer_{roi}_{year}.tif')
    dc_autumn, _, _ = geotiff_to_datacube(f'{folder_composites}seasonal_composite_autumn_{roi}_{year}.tif')

    # using lists and dictionary to avoid slow iteration with pd data frame
    x_list = list(df['X']); x_coords.extend(x_list)
    y_list = list(df['Y']); y_coords.extend(y_list)
    lc_list = list(df['land_cover']); land_covers.extend(lc_list)
    
    features = np.zeros((nrows,len(feature_names)))

    # unpack geotranform
    xOrigin = geotransform[0]
    yOrigin = geotransform[3]
    pixelWidth = geotransform[1]
    pixelHeight = -geotransform[5]
    
    for i, (x_coord,y_coord) in enumerate(zip(x_list,y_list)):
    
        # computing column and row indices
        icol = int((x_coord - xOrigin) / pixelWidth)
        irow = int((yOrigin - y_coord ) / pixelHeight)
    
        features[i,:] = dc_composite[irow,icol,:]

    dataset = pd.DataFrame(data=features,columns=feature_names)
    dataset['roi'] = roi
    datasets.append(dataset)
    
data = pd.concat(datasets,axis=0)
data['X'] = x_coords
data['Y'] = y_coords
data['land_cover'] = land_covers

# rearrange columns
data = data[['roi','X','Y','land_cover',*feature_names]]

data.to_csv(f'{folder_output}seasonal_composite_labeled_as{yearAS}.csv', encoding='utf-8', index=False)
data.head()    